<a href="https://colab.research.google.com/github/ASR-X/ManuScripted/blob/main/backend/Essay_Inspiration_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization Code

In [ ]:
#@title
%tensorflow_version 1.x

!pip install fastapi pyngrok uvicorn nest-asyncio google-cloud-texttospeech gpt-2-simple
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip /content/ngrok-stable-linux-amd64.zip
!./ngrok authtoken 1ooUEkPs7SzF16HgLhHjnsrx8SQ_4zUJGFX5gjWgFj8MH1gFA

TensorFlow 1.x selected.
--2021-03-21 16:26:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.197.133.26, 52.70.180.11, 34.205.198.58, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.197.133.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  14.1MB/s    in 0.9s    

2021-03-21 16:26:19 (14.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  /content/ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/My First Project-ca53a835bffe.json"
def synthesize_text(text):
    from google.cloud import texttospeech
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Wavenet-F",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)#LINEAR16)
    response = client.synthesize_speech(request={"input": input_text, "voice": voice, "audio_config": audio_config})
    with open("output.mp3", "wb") as out:
        out.write(response.audio_content)

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, 3GB on disk.
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook.

In [ ]:
#@title
%%capture
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2 
from datetime import datetime
import tensorflow as tf
from google.colab import files
model_size='124M' 
model_name = model_size


In [ ]:
#@title
gpt2.download_gpt2(model_name=model_name)


Fetching checkpoint: 1.05Mit [00:00, 569Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 917kit/s]
Fetching hparams.json: 1.05Mit [00:00, 755Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:   1%|                 | 3.15M/498M [00:01<05:14, 1.57Mit/s]

In [ ]:
"""#@title
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess(threads=1)
gpt2.load_gpt2(sess, model_name=model_name)
"""

In [ ]:
"""#@title
def inspiration(prompt):
  gpt2.generate(sess,
                    model_name=model_name,
                    prefix=prompt,
                    length=150,
                    temperature=0.75,
                    top_p=0.9,
                    nsamples=3,
                    batch_size=3,
                    return_as_list=True
  )
"""

In [ ]:
#@title
"""
global sess
tf.reset_default_graph()
sess = gpt2.start_tf_sess(threads=1)
gpt2.load_gpt2(sess, model_name=model_name)
global graph
graph = tf.get_default_graph() 
global pog 
pog=graph"""

# Inspiration Flask Server

In [ ]:
flask="""import json
from flask import Flask,request, send_file
from flask_ngrok import run_with_ngrok
import re

app = Flask(__name__)
run_with_ngrok(app) 


@app.route("/")
def root():
    return 'test -- alive'



@app.route('/inspire', methods=['POST'])
def getInspiration():
    global sess
    content = request.json
    g=content["ops"][0]["insert"]#get essay
    print(g)
    global pog 

    graph = pog
    with graph.as_default():
      res=gpt2.generate(sess,
                      model_name=model_name,
                      prefix=g,
                      length=150,
                      temperature=0.75,
                      top_p=0.9,
                      nsamples=3,
                      batch_size=3,
                      return_as_list=True
      )#get 3 samples
    ret=[]

    for r in res:
      re.sub(r"(?<=[a-z])\r?\n"," ", r)

      ret.append(r[len(g):])#trim off input
      
    print(ret)
    rett=[]
    for r in ret:
      print(r)

      try:
        rett.append(re.findall("^.*?[.!?](?=\s[A-Z]|\s?$)(?!.*\))",r)[0])#get first sentence
      except:
        rett.append(r)
    return "\n\n".join(rett)#seperated by double linebreak


if __name__ == '__main__':
    app.run()"""

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi.responses import FileResponse
import gpt_2_simple as gpt2
import tensorflow as tf
import re

class EssayIn(BaseModel):
    essay : str

class ScoreOut(BaseModel):
    in1: str
    in2: str
    in3: str



app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins='*',
    allow_credentials=True,
    allow_methods='*',
    allow_headers='*',
)


tf.reset_default_graph()
sess = gpt2.start_tf_sess(threads=1)
gpt2.load_gpt2(sess, model_name=model_name)

@app.api_route("/", methods=["GET", "POST"])
async def root():
    return 'alive'

@app.post('/inspire', response_model=ScoreOut)
async def getInspiration(essayin: EssayIn):
    global sess
    content = essayin.essay
    g=content#["ops"][0]["insert"]#get essay
    print(g)

    res=gpt2.generate(sess,
                      model_name=model_name,
                      prefix=g,
                      length=150,
                      temperature=0.75,
                      top_p=0.9,
                      nsamples=3,
                      batch_size=3,
                      return_as_list=True
      )#get 3 samples
    ret=[]

    for r in res:
      re.sub(r"(?<=[a-z])\r?\n"," ", r)

      ret.append(r[len(g):])#trim off input
      
    print(ret)
    rett=[]
    for r in ret:
      print(r)

      try:
        rett.append(re.findall("^.*?[.!?](?=\s[A-Z]|\s?$)(?!.*\))",r)[0])#get first sentence
      except:
        rett.append(r)
    return {"in1":rett[0],"in2":rett[1],"in3":rett[2]}#seperated by double linebreak

@app.post("/tts")
async def getSpeech():
    content = request.data#request.json
    g=content.decode("utf-8")#g=content["ops"][0]["insert"]
    print(synthesize_text(g))
    return FileResponse("/content/output.mp3", media_type='audio/mpeg',filename=file_name)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(80, bind_tls=True)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

# Convert DOCX to HTML


In [ ]:
!pip install fastapi nest-asyncio flask_ngrok mammoth
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip /content/ngrok-stable-linux-amd64.zip
#!./ngrok authtoken 1ooUEkPs7SzF16HgLhHjnsrx8SQ_4zUJGFX5gjWgFj8MH1gFA

In [ ]:
import json
from flask import Flask,request
from flask_ngrok import run_with_ngrok
import re
import base64

app = Flask(__name__)
run_with_ngrok(app) 

@app.route('/convertdocx', methods=['POST'])
def convertDocx():
      print(base64.b64decode(request.json['doc']))

if __name__ == '__main__':
    app.run()

Parse All Items into Labels and Data

In [ ]:
import requests

URL = "https://v1.nocodeapi.com/asrx/webflow/arvZVdciEcIdtZsO"
r = requests.get(url = URL)

print(r.json())